In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("../data/final_dataset.csv")

In [3]:
df['DateTime'] = pd.to_datetime(df['Date'] + ' ' + df['Time'], format='%d/%m/%Y %H:%M:%S')

# Setting the Date_Time as index
df.set_index('DateTime', inplace=True)



In [15]:
df1 = df.iloc[0: 5000].copy()

In [5]:
grouped = df.groupby([df.index.floor('h'), 'Classroom Type'])

In [8]:
groups = [group for _, group in grouped]

In [9]:
for group in groups[99:102]:
    print(group[['Date', "Time", "Classroom Type"]])

                           Date     Time             Classroom Type
DateTime                                                           
2023-02-15 08:00:11  15/02/2023  8:00:11  Container With Insulation
2023-02-15 08:00:57  15/02/2023  8:00:57  Container With Insulation
2023-02-15 08:01:44  15/02/2023  8:01:44  Container With Insulation
2023-02-15 08:02:30  15/02/2023  8:02:30  Container With Insulation
2023-02-15 08:03:16  15/02/2023  8:03:16  Container With Insulation
...                         ...      ...                        ...
2023-02-15 08:55:44  15/02/2023  8:55:44  Container With Insulation
2023-02-15 08:56:31  15/02/2023  8:56:31  Container With Insulation
2023-02-15 08:57:17  15/02/2023  8:57:17  Container With Insulation
2023-02-15 08:58:04  15/02/2023  8:58:04  Container With Insulation
2023-02-15 08:58:50  15/02/2023  8:58:50  Container With Insulation

[76 rows x 3 columns]
                           Date     Time      Classroom Type
DateTime                        

In [10]:
df.columns

Index(['Date', 'Time', 'Classroom Type', 'Classroom Category', 'Room No',
       'Device Code', 'School No', 'Grade', 'Measured T', 'Measured RH',
       'Measured CO2', 'Measured PM1.0', 'Measured PM2.5', 'Measured PM10',
       'School Day', 'School Hours', 'Break Time', 'Power On',
       'Outdoor Temperature', 'TmStamp', 'TrackerWM_Avg', 'Tracker2WM_Avg',
       'ShadowWM_Avg', 'SunWM_Avg', 'ShadowbandWM_Avg', 'DNICalc_Avg',
       'AirTC_Avg', 'RH', 'WS_ms_S_WVT', 'WindDir_D1_WVT', 'WindDir_SD1_WVT',
       'BP_mB_Avg', 'UVA_Avg', 'UVB_Avg', 'BattV_Min', 'Batt24V_Min'],
      dtype='object')

In [11]:
ignore_cols = [
    "Date",
    "Time",
    "TmStamp",
    "Classroom Type"
]

cat_cols = [
    "Room No",
    "Device Code",
    "School No",
    "Grade",
    "School Day",
    "School Hours",
    "Break Time",
    "Power On"
]

cont_cols = list(set(list(df.columns)) - set(ignore_cols) - set(cat_cols))

print(ignore_cols)
print(cat_cols)
print(cont_cols)

['Date', 'Time', 'TmStamp', 'Classroom Type']
['Room No', 'Device Code', 'School No', 'Grade', 'School Day', 'School Hours', 'Break Time', 'Power On']
['ShadowbandWM_Avg', 'Measured PM1.0', 'TrackerWM_Avg', 'Measured PM10', 'Batt24V_Min', 'AirTC_Avg', 'Outdoor Temperature', 'WindDir_D1_WVT', 'BattV_Min', 'BP_mB_Avg', 'Measured T', 'Measured PM2.5', 'SunWM_Avg', 'ShadowWM_Avg', 'Tracker2WM_Avg', 'Measured CO2', 'RH', 'WS_ms_S_WVT', 'WindDir_SD1_WVT', 'Measured RH', 'UVA_Avg', 'Classroom Category', 'DNICalc_Avg', 'UVB_Avg']


In [ ]:
def flatten_aggregate(group):
    aggregation = {}

    for col in group.columns:
        if col in ignore_cols:
            aggregation[col] = group[col].values[-1]
        elif col in cat_cols:
            aggregation[col] = group[col].mode()[0]  # Most frequent
        elif col in cont_cols:
            aggregation[f'{col}_mean'] = group[col].mean()
            aggregation[f'{col}_min'] = group[col].min()
            aggregation[f'{col}_max'] = group[col].max()
        else:
            print("Oops : ", col)
             
    return pd.Series(aggregation)